In [1]:
from osgeo import gdal
import numpy
from numpy.core.umath import add, subtract


class MDimage(object):
    def __init__(self, filepath):
        self.filepath = filepath
        self.ds = gdal.Open(filepath, gdal.GA_ReadOnly)
        self.iminfo = dict()
        self.iminfo['bandnum'] = self.ds.RasterCount
        self.iminfo['cols'] = self.ds.RasterXSize
        self.iminfo['rows'] = self.ds.RasterYSize
        self.iminfo['originX'] = self.ds.GetGeoTransform()[0]
        self.iminfo['originY'] = self.ds.GetGeoTransform()[3]
        self.iminfo['pixelWidth'] = self.ds.GetGeoTransform()[1]
        self.iminfo['pixelHeight'] = self.ds.GetGeoTransform()[5]

class Index(MDimage):
    def __init__(self, filepath):
        self.filepath = filepath
        super(Index, self).__init__(filepath)

    def Band2Array(self,factor,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        self.fill=fill
        array = band.ReadAsArray(0, 0, cols, rows)
        #array [array <-100] =fill
        #array = numpy.where(array ==fill, fill , array * 0.0001)
        array= array.reshape(cols*rows)
        return  array

    def trtData(self,evi,lswi,dvel):
        self.evi=evi
        self.lswi=lswi
        self.dvel=dvel
        X=[]
        for i in range(len(evi)):
            F=[]
            F.append(evi[i])
            F.append(lswi[i])
            F.append(dvel[i])
            #F.append(subtract(evi[i],lswi[i]))
            X.append(F)
        return X

    def trainResult(self,flood):
        self.flood=flood
        Y=[]
        for i in range(len(flood)):
            Y.append(flood[i])
        return Y

    def index2Array(self,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        self.fill=fill
        array = band.ReadAsArray(0, 0, cols, rows)
        array = array.reshape(cols * rows)
        return  array

    def csv2Array(self,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        path = "/home/faizan/Desktop/full/result.csv"
        data = numpy.genfromtxt(path, dtype=float, delimiter=',')
        array = data.reshape(cols * rows)
        return  array


    def WriteArrayAsImage(self, out_fname, outArray):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        driver = self.ds.GetDriver()
        self.outArray = outArray
        outArray= outArray.reshape([rows,cols])
        outDS = driver.Create(out_fname, cols, rows, 1, gdal.GDT_Float32)
        outDS.SetGeoTransform(self.ds.GetGeoTransform())
        outDS.SetProjection(self.ds.GetProjection())
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(outArray)
        outDS = None
        del outDS, outBand

In [2]:

import numpy as np
from sklearn import svm
from osgeo import gdal
from sklearn.externals import joblib

fill=-999
path="/home/faizan/Desktop/full/"

#assigning names training data

Tevi=path+"evi_t.tif"
Tdvel=path+"dvel_t.tif"
Tlswi=path+"lswi_t.tif"
Tflood=path+"flood_t.tif"

#assigning names test data
evi=path+"A2014257.evi.tif"
dvel=path+"A2014257.dvel.tif"
lswi=path+"A2014257.lswi.tif"
Oflood = path+"A2014257.machineflood.tif"

#loading image training
limage = Index(Tevi)
cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
EVI_t = (limage.index2Array(fill))

limage = Index(Tlswi)
LSWI_t = (limage.index2Array(fill))

limage = Index(Tdvel)
DVEL_t = (limage.index2Array(fill))

limage = Index(Tflood)
Flood_t = (limage.index2Array(fill))

# arranging training data
X = (limage.trtData(EVI_t, LSWI_t, DVEL_t))
Y = (limage.trainResult(Flood_t))
# SVM machine learning
#clf = svm.SVC()
#clf = clf.fit(X,Y)

#save model

#joblib.dump(clf, '/home/faizan/Desktop/full/trained.pkl')
clf = joblib.load('/home/faizan/Desktop/full/model.pkl') 
"""
#del EVI_t, LSWI_t, DVEL_t, Flood_t, X, Y
#Test data

limage = Index(evi)
cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
EVI = (limage.index2Array(fill))

limage = Index(lswi)
LSWI = (limage.index2Array(fill))

limage = Index(dvel)
DVEL = (limage.index2Array(fill))

# arranging test data
data = (limage.trtData(EVI, LSWI, DVEL))

#output result
flood = clf.predict(data)
#Get Flood as tiff
limage.WriteArrayAsImage(Oflood, flood)
del flood, EVI, LSWI, DVEL, data
"""

"\n#del EVI_t, LSWI_t, DVEL_t, Flood_t, X, Y\n#Test data\n\nlimage = Index(evi)\ncols = limage.iminfo['cols']\n# Get pixel row number\nrows = limage.iminfo['rows']\nEVI = (limage.index2Array(fill))\n\nlimage = Index(lswi)\nLSWI = (limage.index2Array(fill))\n\nlimage = Index(dvel)\nDVEL = (limage.index2Array(fill))\n\n# arranging test data\ndata = (limage.trtData(EVI, LSWI, DVEL))\n\n#output result\nflood = clf.predict(data)\n#Get Flood as tiff\nlimage.WriteArrayAsImage(Oflood, flood)\ndel flood, EVI, LSWI, DVEL, data\n"

In [3]:
from sklearn.cross_validation import cross_val_score
score = cross_val_score(clf, X, Y)
score.mean()

0.98328225729624119

In [4]:
limage = Index(Oflood)
Flood_o = (limage.index2Array(fill))

In [5]:
def cMatrix(actual, test):
    F_F=0
    F_N=0
    F_fi=0
    N_F =0
    N_N =0
    N_fi=0
    Fi_F=0
    Fi_N=0
    Fi_fi=0

    for i in range(len(actual)):
        if actual[i]==test[i]:
            if actual[i]==0:
                N_N += 1
            elif actual[i]==1:
                F_F += 1
            else:
                Fi_fi += 1
        else:
            if actual[i]==0:
                if test[i]==1:
                    N_F += 1
                else:
                    N_fi+= 1
            elif actual[i]==1:
                if test[i]==0:
                    F_N +=1
                else:
                    F_fi +=1 
            else:
                if test[i]==1:
                    Fi_F +=1
                else:
                    Fi_N +=1

    LIST= [F_F,F_N,F_fi,N_F,N_N,
           N_fi,Fi_F,Fi_N,Fi_fi]

    C_matrix = numpy.array(LIST).reshape(3,3)
    return C_matrix

In [6]:
cMat= cMatrix(Flood_t, Flood_o)
cMat

array([[  1739,  87121,  42643],
       [  4451, 192430,  94500],
       [  4582, 205619, 100327]])